In [1]:
import pymn
import pandas as pd
import matplotlib.pyplot as plt
import scanpy as sc
import anndata as ad
import numpy as np
import itertools
import scipy.stats
import seaborn as sns
import scgen 
sc.settings.verbosity = 3   
sc.set_figure_params(figsize = (15,15),facecolor = 'white')

Global seed set to 0
/home/passala/miniconda3/envs/Single_cell_data_fix/lib/python3.9/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/home/passala/miniconda3/envs/Single_cell_data_fix/lib/python3.9/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


In [5]:
tf_list = pd.read_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/Initialization/zma_TF2fam2mot.txt', sep = '\t')
tf_list = tf_list['Zm00001d050368']
tf_list

0       Zm00001d037159
1       Zm00001d037425
2       Zm00001d020951
3       Zm00001d032239
4       Zm00001d051861
             ...      
2142    Zm00001d053687
2143    Zm00001d048105
2144    Zm00001d039118
2145    Zm00001d003284
2146    Zm00001d000110
Name: Zm00001d050368, Length: 2147, dtype: object

In [6]:
#tf_list.to_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/INPUTS/TF_list.txt', index= False, sep = '\t', header = False)

In [2]:
norm_data = sc.read_h5ad('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Post_review_Dataset/faster_reading_full_data.h5ad')

In [3]:
cell_identity = pd.read_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Post_review_Dataset/Meta_Maize_Sorghum_Setaria_Nuclei_Cells_SCT_OrthologousgenesGoodname_Sept2022.csv',index_col = 0)
norm_data.obs = cell_identity

In [4]:
norm_data.obs = norm_data.obs.rename(columns = {'orig.ident':'Batch','Annotation':'Cell Cluster Annotation','type':'Type'})

In [5]:
norm_data.var['ZM'] = norm_data.var_names.str.startswith('Zm')
norm_data = norm_data[:, norm_data.var['ZM'] == True] 

In [6]:
cell_type_list = norm_data.obs['Cell Cluster Annotation'].unique()

In [7]:
fake_cluster_id = np.arange(len(cell_type_list))
fake_cluster_id

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18])

In [8]:
cluster_2_ident = pd.DataFrame(data = zip(fake_cluster_id,cell_type_list))
cluster_2_ident

,0,1
0,0,Cortex_4
1,1,Stele_2
2,2,Cortex_1
3,3,Epidermis_2
4,4,Young_Stele_2
5,5,Stele_1
6,6,Epidermis_1
7,7,Cortex_3
8,8,Pericycle
9,9,Epidermis_3


In [30]:
norm_data.obs['Cell ID']= norm_data.obs.index
norm_data.obs

/tmp/ipykernel_3164068/3738641279.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  norm_data.obs['Cell ID']= norm_data.obs.index


,Batch,nCount_RNA,nFeature_RNA,nCount_SCT,nFeature_SCT,integrated_snn_res.0.6,seurat_clusters,Species,Cell Cluster Annotation,Type,Cell ID
Maize_Cell1_AAACCTGAGAGCTTCT,Maize_Cell_1,7907,3206,13445,3280,6,6,Maize,Cortex_4,Cell,Maize_Cell1_AAACCTGAGAGCTTCT
Maize_Cell1_AAACCTGAGTGATCGG,Maize_Cell_1,11556,3579,14008,3580,7,7,Maize,Stele_2,Cell,Maize_Cell1_AAACCTGAGTGATCGG
Maize_Cell1_AAACCTGCACTGTCGG,Maize_Cell_1,19725,4690,15803,4688,1,1,Maize,Cortex_1,Cell,Maize_Cell1_AAACCTGCACTGTCGG
Maize_Cell1_AAACCTGGTGTGAAAT,Maize_Cell_1,17272,4355,15469,4354,6,6,Maize,Cortex_4,Cell,Maize_Cell1_AAACCTGGTGTGAAAT
Maize_Cell1_AAACCTGTCTCAAGTG,Maize_Cell_1,11246,3548,13956,3546,14,14,Maize,Epidermis_2,Cell,Maize_Cell1_AAACCTGTCTCAAGTG
...,...,...,...,...,...,...,...,...,...,...,...
Set_Cells_3_TTTGGTTTCTGGGTCG,Set_Cells_3,9192,2954,21040,3212,14,14,Setaria,Epidermis_2,Cell,Set_Cells_3_TTTGGTTTCTGGGTCG
Set_Cells_3_TTTGTTGAGGCTCACC,Set_Cells_3,27048,4655,23549,4654,16,16,Setaria,Trichoblast,Cell,Set_Cells_3_TTTGTTGAGGCTCACC
Set_Cells_3_TTTGTTGCAACCAGAG,Set_Cells_3,21855,5009,22238,5009,17,17,Setaria,Phloem,Cell,Set_Cells_3_TTTGTTGCAACCAGAG
Set_Cells_3_TTTGTTGCAGTCTACA,Set_Cells_3,9015,2669,20346,3006,3,3,Setaria,LRC,Cell,Set_Cells_3_TTTGTTGCAGTCTACA


In [31]:
cell_2_clusters = norm_data.obs.merge(left_on='Cell Cluster Annotation',right = cluster_2_ident, right_on= 1)

In [32]:
cell_2_clusters

,Batch,nCount_RNA,nFeature_RNA,nCount_SCT,nFeature_SCT,integrated_snn_res.0.6,seurat_clusters,Species,Cell Cluster Annotation,Type,Cell ID,0,1
0,Maize_Cell_1,7907,3206,13445,3280,6,6,Maize,Cortex_4,Cell,Maize_Cell1_AAACCTGAGAGCTTCT,0,Cortex_4
1,Maize_Cell_1,17272,4355,15469,4354,6,6,Maize,Cortex_4,Cell,Maize_Cell1_AAACCTGGTGTGAAAT,0,Cortex_4
2,Maize_Cell_1,34529,5645,16170,4889,6,6,Maize,Cortex_4,Cell,Maize_Cell1_AAAGTAGCACGAAATA,0,Cortex_4
3,Maize_Cell_1,29551,6059,16385,5734,6,6,Maize,Cortex_4,Cell,Maize_Cell1_AAAGTAGGTTACGCGC,0,Cortex_4
4,Maize_Cell_1,12301,3742,14118,3741,6,6,Maize,Cortex_4,Cell,Maize_Cell1_AACGTTGAGGCATTGG,0,Cortex_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47531,Set_Cells_3,18623,4651,21624,4650,15,15,Setaria,Columella,Cell,Set_Cells_3_TTGTGTTCATGAAGGC,18,Columella
47532,Set_Cells_3,9025,3383,20208,3656,15,15,Setaria,Columella,Cell,Set_Cells_3_TTGTTCAAGCGCGTTC,18,Columella
47533,Set_Cells_3,10876,4089,21373,4154,15,15,Setaria,Columella,Cell,Set_Cells_3_TTTCACACATCAGCGC,18,Columella
47534,Set_Cells_3,5996,2958,19471,3765,15,15,Setaria,Columella,Cell,Set_Cells_3_TTTCCTCCAAGAAATC,18,Columella


In [33]:
cell_2_clusters = cell_2_clusters.rename(columns = {0:'Forced Clusters'})
cell_2_clusters

,Batch,nCount_RNA,nFeature_RNA,nCount_SCT,nFeature_SCT,integrated_snn_res.0.6,seurat_clusters,Species,Cell Cluster Annotation,Type,Cell ID,Forced Clusters,1
0,Maize_Cell_1,7907,3206,13445,3280,6,6,Maize,Cortex_4,Cell,Maize_Cell1_AAACCTGAGAGCTTCT,0,Cortex_4
1,Maize_Cell_1,17272,4355,15469,4354,6,6,Maize,Cortex_4,Cell,Maize_Cell1_AAACCTGGTGTGAAAT,0,Cortex_4
2,Maize_Cell_1,34529,5645,16170,4889,6,6,Maize,Cortex_4,Cell,Maize_Cell1_AAAGTAGCACGAAATA,0,Cortex_4
3,Maize_Cell_1,29551,6059,16385,5734,6,6,Maize,Cortex_4,Cell,Maize_Cell1_AAAGTAGGTTACGCGC,0,Cortex_4
4,Maize_Cell_1,12301,3742,14118,3741,6,6,Maize,Cortex_4,Cell,Maize_Cell1_AACGTTGAGGCATTGG,0,Cortex_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47531,Set_Cells_3,18623,4651,21624,4650,15,15,Setaria,Columella,Cell,Set_Cells_3_TTGTGTTCATGAAGGC,18,Columella
47532,Set_Cells_3,9025,3383,20208,3656,15,15,Setaria,Columella,Cell,Set_Cells_3_TTGTTCAAGCGCGTTC,18,Columella
47533,Set_Cells_3,10876,4089,21373,4154,15,15,Setaria,Columella,Cell,Set_Cells_3_TTTCACACATCAGCGC,18,Columella
47534,Set_Cells_3,5996,2958,19471,3765,15,15,Setaria,Columella,Cell,Set_Cells_3_TTTCCTCCAAGAAATC,18,Columella


In [34]:
cell_2_clusters = cell_2_clusters[['Cell ID','Forced Clusters']]

In [35]:
cell_2_clusters

,Cell ID,Forced Clusters
0,Maize_Cell1_AAACCTGAGAGCTTCT,0
1,Maize_Cell1_AAACCTGGTGTGAAAT,0
2,Maize_Cell1_AAAGTAGCACGAAATA,0
3,Maize_Cell1_AAAGTAGGTTACGCGC,0
4,Maize_Cell1_AACGTTGAGGCATTGG,0
...,...,...
47531,Set_Cells_3_TTGTGTTCATGAAGGC,18
47532,Set_Cells_3_TTGTTCAAGCGCGTTC,18
47533,Set_Cells_3_TTTCACACATCAGCGC,18
47534,Set_Cells_3_TTTCCTCCAAGAAATC,18


In [ ]:
#cell_2_clusters.to_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/INPUTS/forced_cells2clusters.txt', index= False, sep = '\t', header = False)

In [29]:
norm_data.obs

,Batch,nCount_RNA,nFeature_RNA,nCount_SCT,nFeature_SCT,integrated_snn_res.0.6,seurat_clusters,Species,Cell Cluster Annotation,Type
Maize_Cell1_AAACCTGAGAGCTTCT,Maize_Cell_1,7907,3206,13445,3280,6,6,Maize,Cortex_4,Cell
Maize_Cell1_AAACCTGAGTGATCGG,Maize_Cell_1,11556,3579,14008,3580,7,7,Maize,Stele_2,Cell
Maize_Cell1_AAACCTGCACTGTCGG,Maize_Cell_1,19725,4690,15803,4688,1,1,Maize,Cortex_1,Cell
Maize_Cell1_AAACCTGGTGTGAAAT,Maize_Cell_1,17272,4355,15469,4354,6,6,Maize,Cortex_4,Cell
Maize_Cell1_AAACCTGTCTCAAGTG,Maize_Cell_1,11246,3548,13956,3546,14,14,Maize,Epidermis_2,Cell
...,...,...,...,...,...,...,...,...,...,...
Set_Cells_3_TTTGGTTTCTGGGTCG,Set_Cells_3,9192,2954,21040,3212,14,14,Setaria,Epidermis_2,Cell
Set_Cells_3_TTTGTTGAGGCTCACC,Set_Cells_3,27048,4655,23549,4654,16,16,Setaria,Trichoblast,Cell
Set_Cells_3_TTTGTTGCAACCAGAG,Set_Cells_3,21855,5009,22238,5009,17,17,Setaria,Phloem,Cell
Set_Cells_3_TTTGTTGCAGTCTACA,Set_Cells_3,9015,2669,20346,3006,3,3,Setaria,LRC,Cell


In [27]:
#cluster_2_ident.to_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/INPUTS/forced_cluster_identities.txt', index= False, sep = '\t')

In [10]:
panda_of_expression_results = pd.DataFrame(index = norm_data.obs.index, columns= norm_data.var.index, data = norm_data.X )
panda_of_expression_results

,Zm00001d027230,Zm00001d027231,Zm00001d027236,Zm00001d027239,Zm00001d027240,Zm00001d027242,Zm00001d027250,Zm00001d027256,Zm00001d027257,Zm00001d027258,...,Zm00001d002687,Zm00001d052418,Zm00001d052397,Zm00001d053127,Zm00001d033915,Zm00001d033266,Zm00001d032469,Zm00001d029568,Zm00001d047419,Zm00001d028222
Maize_Cell1_AAACCTGAGAGCTTCT,0.0,0.817443,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Maize_Cell1_AAACCTGAGTGATCGG,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Maize_Cell1_AAACCTGCACTGTCGG,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Maize_Cell1_AAACCTGGTGTGAAAT,0.0,0.769156,0.0,0.0,0.000000,0.0,0.0,0.456774,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Maize_Cell1_AAACCTGTCTCAAGTG,0.0,0.000000,0.0,0.0,0.636156,0.0,0.0,0.000000,0.636156,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Set_Cells_3_TTTGGTTTCTGGGTCG,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Set_Cells_3_TTTGTTGAGGCTCACC,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Set_Cells_3_TTTGTTGCAACCAGAG,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Set_Cells_3_TTTGTTGCAGTCTACA,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
norm

In [11]:
filtered_non_expressed_genes = panda_of_expression_results.loc[:,(panda_of_expression_results.sum()>1)]



In [12]:
filtered_non_expressed_genes

,Zm00001d027230,Zm00001d027231,Zm00001d027236,Zm00001d027239,Zm00001d027240,Zm00001d027242,Zm00001d027250,Zm00001d027256,Zm00001d027257,Zm00001d027258,...,Zm00001d027490,Zm00001d006416,Zm00001d037737,Zm00001d038726,Zm00001d023869,Zm00001d035907,Zm00001d036242,Zm00001d002772,Zm00001d033915,Zm00001d029568
Maize_Cell1_AAACCTGAGAGCTTCT,0.0,0.817443,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Maize_Cell1_AAACCTGAGTGATCGG,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Maize_Cell1_AAACCTGCACTGTCGG,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Maize_Cell1_AAACCTGGTGTGAAAT,0.0,0.769156,0.0,0.0,0.000000,0.0,0.0,0.456774,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Maize_Cell1_AAACCTGTCTCAAGTG,0.0,0.000000,0.0,0.0,0.636156,0.0,0.0,0.000000,0.636156,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Set_Cells_3_TTTGGTTTCTGGGTCG,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Set_Cells_3_TTTGTTGAGGCTCACC,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Set_Cells_3_TTTGTTGCAACCAGAG,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Set_Cells_3_TTTGTTGCAGTCTACA,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
transposed_matrix = filtered_non_expressed_genes.T
transposed_matrix.index.name = 'geneid'

In [18]:
transposed_matrix

,Maize_Cell1_AAACCTGAGAGCTTCT,Maize_Cell1_AAACCTGAGTGATCGG,Maize_Cell1_AAACCTGCACTGTCGG,Maize_Cell1_AAACCTGGTGTGAAAT,Maize_Cell1_AAACCTGTCTCAAGTG,Maize_Cell1_AAACGGGAGGAGTTTA,Maize_Cell1_AAACGGGGTAAAGGAG,Maize_Cell1_AAACGGGTCAAGATCC,Maize_Cell1_AAAGATGAGAGGTTGC,Maize_Cell1_AAAGATGCAGAGCCAA,...,Set_Cells_3_TTTGGAGGTTCGGCTG,Set_Cells_3_TTTGGAGTCTGTAACG,Set_Cells_3_TTTGGTTAGCCGATCC,Set_Cells_3_TTTGGTTAGTCACTGT,Set_Cells_3_TTTGGTTGTTACCCAA,Set_Cells_3_TTTGGTTTCTGGGTCG,Set_Cells_3_TTTGTTGAGGCTCACC,Set_Cells_3_TTTGTTGCAACCAGAG,Set_Cells_3_TTTGTTGCAGTCTACA,Set_Cells_3_TTTGTTGGTGGTCTCG
geneid,,,,,,,,,,,,,,,,,,,,,
Zm00001d027230,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00000,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zm00001d027231,0.817443,0.0,0.0,0.769156,0.000000,0.0,0.38086,0.63532,0.586811,0.586757,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zm00001d027236,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00000,0.00000,0.000000,0.586757,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zm00001d027239,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00000,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zm00001d027240,0.000000,0.0,0.0,0.000000,0.636156,0.0,0.00000,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zm00001d035907,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00000,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zm00001d036242,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00000,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zm00001d002772,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00000,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
#transposed_matrix.to_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/INPUTS/full_dataset_matrix.txt', index= False, sep = '\t')

In [23]:
csv_version_markers = pd.read_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/INPUTS/maize_high_purity_allMarkers.txt', sep = '\t')
csv_version_markers

,geneID,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,cluster,gene
0,Zm00001d007560,2.290575e-18,0.295872,0.094,0.036,1.390562e-13,5,Zm00001d007560
1,Zm00001d052041,1.459176e-219,0.830747,0.477,0.081,8.858364e-215,2,Zm00001d052041
2,Zm00001d025018,4.380989e-200,0.912269,0.498,0.096,2.659611e-195,2,Zm00001d025018
3,Zm00001d009060,2.911081e-160,0.542515,0.400,0.072,1.767259e-155,2,Zm00001d009060
4,Zm00001d004340,4.143612e-122,0.447059,0.319,0.060,2.515504e-117,2,Zm00001d004340
...,...,...,...,...,...,...,...,...
607,Zm00001d0167301,1.119558e-38,0.281407,0.323,0.084,6.796614e-34,9,Zm00001d016730
608,Zm00001d010079,3.426795e-37,0.271706,0.343,0.093,2.080339e-32,9,Zm00001d010079
609,Zm00001d0328812,1.532764e-30,0.350321,0.227,0.053,9.305106e-26,9,Zm00001d032881
610,Zm00001d038451,7.070888e-25,0.263717,0.291,0.095,4.292594e-20,9,Zm00001d038451


In [16]:
# csv_version_markers = csv_version_markers.merge(left_on='cluster',right = cluster_2_ident, right_on= 1)
# csv_version_markers = csv_version_markers.drop(columns= ['cluster',1])
# csv_version_markers
# csv_version_markers = csv_version_markers.rename(columns = {0:'cluster'})
# csv_version_markers
# csv_version_markers = csv_version_markers[['geneID','p_val','avg_log2FC','pct.1','pct.2','p_val_adj','cluster','gene']]
# csv_version_markers

In [22]:
#csv_version_markers.to_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/INPUTS/maize_high_purity_allMarkers.txt', index= False, sep = '\t')